In [31]:
# WORK IN PROGRESS



# various imports
import numpy as np
from matplotlib import pyplot as plt

#matrix_dim
d = 4
lower_diag = 1/2
main_diag = 2
upper_diag = 1/2

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)
ld = lower_diag*np.ones(d-1)
md = main_diag*np.ones(d)
ud = upper_diag*np.ones(d-1)
A = tridiag(ld, md, ud)
print('matrix A is: \n' + str(A))

print('the eigenvalues of A are: \n' + str(np.linalg.eig(A)[0]))

# exact solution
x = np.ones([d,1])
print('we set the exact solution x: \n' + str(x))
b = A.dot(x)
print('and the vector b is: \n' +str(b))

matrix A is: 
[[2.  0.5 0.  0. ]
 [0.5 2.  0.5 0. ]
 [0.  0.5 2.  0.5]
 [0.  0.  0.5 2. ]]
the eigenvalues of A are: 
[1.19098301 1.69098301 2.80901699 2.30901699]
we set the exact solution x: 
[[1.]
 [1.]
 [1.]
 [1.]]
and the vector b is: 
[[2.5]
 [3. ]
 [3. ]
 [2.5]]


In [32]:
# Imports for Qiskit
from qiskit.compiler import transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT

def qpe(circ, clock, input, c_qubits):
    circ.barrier()

    for j in range(c_qubits):
        for _ in range(2**j):
            # perform e^(iAt) 2^j times controlled by clock[j] to input qubits
            circ.cu(   ''  , clock[j], input)
    circ.barrier()
    
    # Perform an inverse QFT on the register holding the eigenvalues
    circ.compose(
    QFT(c_qubits, inverse=True), clock,
    inplace=True)
    
def inv_qpe(circ, clock, input, c_qubits):
    
    # Perform a QFT on the register holding the eigenvalues
    circ.compose(
    QFT(c_qubits, inverse=False), clock,
    inplace=True)

    circ.barrier()

    for j in range(c_qubits):
        for _ in range(2**j):
            # perform e^(-iAt) 2^j times controlled by clock[j] to input qubits
            circ.cu(np.pi/2, np.pi/2, -np.pi/2, -3*np.pi/4, clock[j], input)

    circ.barrier()

def hhl(circ, ancilla, clock, input, c_qubits):
    
    qpe(circ, clock, input, c_qubits)

    circ.barrier()
    
    circ.cry(2*np.asin(1), clock[c_qubits-2], ancilla)



    #deferred measurement principle
    #circuit.measure(ancilla, measurement[0])
    circ.barrier()
    inv_qpe(circ, clock, input, c_qubits)

def hhl_complete(circ, ancilla, clock, input, measurement, b, c_qubits): # State preparation. 

    circ.initialize(b, c_qubits+np.log2(d))

    circ.barrier()

    # Perform a Hadamard Transform on all clock qubits
    circ.h(clock)

    hhl(circ, ancilla, clock, input, c_qubits)

    # Perform a Hadamard Transform on all clock qubits
    circ.h(clock)

    circ.barrier()


    circ.measure(ancilla, measurement[0])
    meas_nums = []
    for j in range(1,d+1):
        meas_nums.append(j)
    circ.measure(input, meas_nums)
    return circ